In [1]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
from urllib.parse import quote

import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert

from datetime import datetime, timedelta

import numpy as np
import statistics
from matplotlib import font_manager, rc 
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import seaborn as sns

#마이너스 표시
mpl.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='Malgun Gothic')

# 파일 전처리

1. 개봉일과 기준일 날짜형식으로 바꾸기
2. 개봉일, 기준일 기준 - 연도, 월 컬럼 각각 추가
3. 매출액과 관객수, 스크린수, 상영횟수가 모두 0인 행 삭제
4. 등급 이상값 확인 후 변경
5. excel로 저장 (데이터형식을 유지하기위함)
 - 파일명 : ./data/완_기간별_일별_170101_to_230509_ver2.xlsx

In [18]:
movie_df = pd.read_csv("./data/완_기간별_일별_170101_to_230509_ver1.csv")
movie_df.head()

,순위,영화명,개봉일,대표국적,국적,제작사,배급사,등급,장르,감독,...,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,기준연도,기준월,개봉연도,개봉월
0,1,너의 이름은.,2017-01-04,일본,일본,NaN,플러스엠 엔터테인먼트,12세이상관람가,"애니메이션,드라마,멜로/로맨스",신카이 마코토,...,2809088900,9745292442,333008,1182465,947,4101,2017,1,2017,1
1,3,모아나,2017-01-12,미국,미국,NaN,월트디즈니컴퍼니코리아 유한책임회사,전체관람가,애니메이션,"론 클레멘츠,존 머스커",...,7738000,9712000,910,1192,6,6,2017,1,2017,1
2,4,패신저스,2017-01-04,미국,미국,NaN,유니버설픽쳐스인터내셔널 코리아(유),12세이상관람가,"어드벤처,SF",모튼 틸덤,...,1107502600,4176443418,129810,504393,608,2272,2017,1,2017,1
3,9,눈의 여왕 3: 눈과 불의 마법대결,2017-01-04,러시아,러시아,NaN,(주)이수C&E,전체관람가,"애니메이션,어드벤처,가족,판타지",알렉세이 트시칠린,...,535436000,1828596400,67605,242171,524,1163,2017,1,2017,1
4,10,사랑하기 때문에,2017-01-04,한국,한국,(주)에이디사공육,(주)넥스트엔터테인먼트월드(NEW),12세이상관람가,"코미디,멜로/로맨스",주지홍,...,450612400,1968850940,53856,246167,542,1533,2017,1,2017,1


In [3]:
# 개봉일과 기준일 날짜형식으로 바꾸기
movie_df['개봉일'] = pd.to_datetime(movie_df['개봉일'])
movie_df['기준일'] = pd.to_datetime(movie_df['기준일'])
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363272 entries, 0 to 363271
Data columns (total 18 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   순위      363272 non-null  int64         
 1   영화명     363272 non-null  object        
 2   개봉일     363272 non-null  datetime64[ns]
 3   대표국적    363272 non-null  object        
 4   국적      363272 non-null  object        
 5   제작사     137977 non-null  object        
 6   배급사     363048 non-null  object        
 7   등급      363258 non-null  object        
 8   장르      362698 non-null  object        
 9   감독      338709 non-null  object        
 10  배우      285173 non-null  object        
 11  기준일     363272 non-null  datetime64[ns]
 12  매출액     363272 non-null  int64         
 13  누적매출액   363272 non-null  int64         
 14  관객수     363272 non-null  int64         
 15  누적관객수   363272 non-null  int64         
 16  스크린수    363272 non-null  int64         
 17  상영횟수    363272 non-null  int6

In [4]:
# 개봉일, 기준일 기준 - 연도, 월 컬럼 각각 추가
movie_df['기준연도'] = movie_df['기준일'].apply(lambda x : x.year)
movie_df['기준월'] = movie_df['기준일'].apply(lambda x : x.month)
movie_df['개봉연도'] = movie_df['개봉일'].apply(lambda x : x.year)
movie_df['개봉월'] = movie_df['개봉일'].apply(lambda x : x.month)
movie_df.head(1)

,순위,영화명,개봉일,대표국적,국적,제작사,배급사,등급,장르,감독,...,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,기준연도,기준월,개봉연도,개봉월
0,1,너의 이름은.,2017-01-04,일본,일본,NaN,플러스엠 엔터테인먼트,12세이상관람가,"애니메이션,드라마,멜로/로맨스",신카이 마코토,...,2809088900,9745292442,333008,1182465,947,4101,2017,1,2017,1


In [5]:
# 매출액, 관객수, 스크린수, 상영횟수 모두가 0인 행 제외
movie_df = movie_df[(movie_df['매출액']!=0)&(movie_df['관객수']!=0)&
                   (movie_df['스크린수']!=0)&(movie_df['상영횟수']!=0)]

In [6]:
len(movie_df)

183006

In [7]:
# 등급 이상 값 확인
movie_df['등급'].unique()

array(['12세이상관람가', '전체관람가', '청소년관람불가', '15세이상관람가', '15세관람가', nan,
       '12세이상관람가,15세이상관람가', '연소자관람불가,청소년관람불가', '12세관람가'], dtype=object)

In [8]:
len(movie_df[movie_df['등급']=='12세이상관람가,15세이상관람가'])
movie_df[movie_df['등급']=='12세이상관람가,15세이상관람가']["영화명"].unique()

array(['라이크 크레이지'], dtype=object)

In [9]:
len(movie_df[movie_df['등급']=='연소자관람불가,청소년관람불가'])
movie_df[movie_df['등급']=='연소자관람불가,청소년관람불가']["영화명"].unique()

array(['죽어도 좋은 경험: 천사여 악녀가 되라'], dtype=object)

In [10]:
len(movie_df[movie_df['등급']=='12세관람가'])
movie_df[movie_df['등급']=='12세관람가']["영화명"].unique()

array(['던전 앤 드래곤: 도적들의 명예'], dtype=object)

In [11]:
# 등급 이상 값 확인 후 변경
movie_df['등급'].replace(['15세관람가', '12세이상관람가,15세이상관람가', '연소자관람불가,청소년관람불가', '12세관람가'],['15세이상관람가', '12세이상관람가', '청소년관람불가', '12세이상관람가'], inplace=True)
movie_df['등급'].unique()

array(['12세이상관람가', '전체관람가', '청소년관람불가', '15세이상관람가', nan], dtype=object)

In [18]:
movie_df.to_excel('./data/완_기간별_일별_170101_to_230509_ver2.xlsx', index=False)

In [19]:
movie_df.to_csv('./data/완_기간별_일별_170101_to_230509_ver2.csv', index=False)